# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import numpy as np
import pandas as pd
import pickle
#import os
from sqlalchemy import create_engine
import re
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean
from sklearn.preprocessing import FunctionTransformer

nltk.download(['punkt','wordnet','averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# load data from database

engine = create_engine('sqlite:///DisasterResponse.db')

df = pd.read_sql_table('df',engine)
X = df['message']
Y = df.iloc[:,4:]

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
Y.isnull().sum()

related                   138
request                   138
offer                     138
aid_related               138
medical_help              138
medical_products          138
search_and_rescue         138
security                  138
military                  138
child_alone               138
water                     138
food                      138
shelter                   138
clothing                  138
money                     138
missing_people            138
refugees                  138
death                     138
other_aid                 138
infrastructure_related    138
transport                 138
buildings                 138
electricity               138
tools                     138
hospitals                 138
shops                     138
aid_centers               138
other_infrastructure      138
weather_related           138
floods                    138
storm                     138
fire                      138
earthquake                138
cold      

In [8]:
X.isnull().sum()

0

In [9]:
Y.fillna(0,inplace=True)

In [10]:
Y.isnull().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    # tokens the text
    tokens = WhitespaceTokenizer().tokenize(text)
    lemmatizers = WordNetLemmatizer()
    
    # clean tokens
    
    process_tokens = []
    
    for token in tokens:
        token = lemmatizers.lemmatize(token).lower().strip('!"#$%\'()*+,-./:;<=>?@[\\]^_,{|}~')
        token = re.sub(r'\[[^.,;:]]*\]','',token)
        # add token to compiled list
        
        if token !='':
            process_tokens.append(token)
            
    return process_tokens
    
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline ([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [14]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
y_pred = pipeline.predict(X_test)

In [16]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.38      0.07      0.11       902
               request       0.00      0.00      0.00        25
                 offer       0.45      0.20      0.28      2178
           aid_related       0.05      0.00      0.00       413
          medical_help       0.12      0.01      0.02       267
      medical_products       0.00      0.00      0.00       135
     search_and_rescue       0.00      0.00      0.00        99
              security       0.00      0.00      0.00       161
              military       0.00      0.00      0.00         0
           child_alone       0.10      0.01      0.02       342
                 water       0.21      0.01      0.01       599
                  food       0.13      0.01      0.01       463
               shelter       0.50      0.01      0.02        91
              clothing       0.00      0.00      0.00       131
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {'vect__ngram_range': ((1,2), (2,2)),
              #'tfidf__use_idf': (True, False)}
              'clf__estimator__n_estimators': [1, 10]}

cv = GridSearchCV(pipeline, parameters)

In [18]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 2), (2, 2)), 'clf__estimator__n_estimators': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
y_pred = cv.predict(X_test)

In [20]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.41      0.06      0.11       902
               request       0.00      0.00      0.00        25
                 offer       0.44      0.21      0.28      2178
           aid_related       0.13      0.00      0.01       413
          medical_help       0.22      0.01      0.01       267
      medical_products       0.00      0.00      0.00       135
     search_and_rescue       0.00      0.00      0.00        99
              security       0.00      0.00      0.00       161
              military       0.00      0.00      0.00         0
           child_alone       0.06      0.00      0.01       342
                 water       0.19      0.01      0.02       599
                  food       0.11      0.00      0.01       463
               shelter       0.00      0.00      0.00        91
              clothing       0.50      0.01      0.02       131
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
def compute_text_lenght(data):
    return np.array([len(text) for text in data]).reshape(-1,1)

In [22]:
pipeline = Pipeline([
    ('features', FeatureUnion([('text', Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                                                 ('tfidf', TfidfTransformer()),
                                                 ])),
                                                 
                              ('lenght', Pipeline([('count', FunctionTransformer(compute_text_lenght,
                                                                                 validate = False))]))])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

In [23]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, ma...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [24]:
y_pred = pipeline.predict(X_test)

In [25]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.41      0.07      0.11       902
               request       0.00      0.00      0.00        25
                 offer       0.45      0.20      0.27      2178
           aid_related       0.00      0.00      0.00       413
          medical_help       0.06      0.00      0.01       267
      medical_products       0.22      0.01      0.03       135
     search_and_rescue       0.00      0.00      0.00        99
              security       0.00      0.00      0.00       161
              military       0.00      0.00      0.00         0
           child_alone       0.09      0.01      0.01       342
                 water       0.12      0.01      0.01       599
                  food       0.07      0.00      0.01       463
               shelter       0.50      0.01      0.02        91
              clothing       0.00      0.00      0.00       131
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [26]:
parameters = {'features__text__vect__ngram_range': ((1,2), (2,2)),
              #'tfidf__use_idf': (True, False)}
              'clf__estimator__n_estimators': [1, 10]}
                 

cv = GridSearchCV(pipeline, parameters)

In [27]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, ma...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text__vect__ngram_range': ((1, 2), (2, 2)), 'clf__estimator__n_estimators': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
y_pred = cv.predict(X_test)

In [29]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.42      0.06      0.11       902
               request       0.00      0.00      0.00        25
                 offer       0.45      0.21      0.28      2178
           aid_related       0.00      0.00      0.00       413
          medical_help       0.14      0.00      0.01       267
      medical_products       0.00      0.00      0.00       135
     search_and_rescue       0.00      0.00      0.00        99
              security       0.00      0.00      0.00       161
              military       0.00      0.00      0.00         0
           child_alone       0.17      0.01      0.01       342
                 water       0.32      0.02      0.03       599
                  food       0.18      0.01      0.02       463
               shelter       0.50      0.01      0.02        91
              clothing       0.00      0.00      0.00       131
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [30]:
print(accuracy_score(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred])))

0.330043651547


### 9. Export your model as a pickle file

In [31]:
pickle.dump(cv, open('model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.